In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
# import local libs
import ecg_tools_lite as et

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [2]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device, time.localtime().tm_mon, "/", time.localtime().tm_mday, "/", time.localtime().tm_year)

Device: cuda 11 / 14 / 2020


In [3]:
ecg_clean = et.load_ecg_file('all_chunks_fs1024_e24.npy')
ecg_noisy = et.load_ecg_file('all_chunks_fs1024_e06.npy')

trainloader = DataLoader( ecg_noisy, shuffle=True, num_workers=2 )
# testloader = DataLoader()

# Testing Autoencoder

In [4]:
###############################
# Seems to be incompatible
###############################
# from sequitur.models import LINEAR_AE
# from sequitur import quick_train 

In [5]:
in_channels = ecg_clean.shape[0]
out_channels = ecg_clean.shape[0]
kernel_size = 1

In [6]:
class Encoder(nn.Module):
    def __init__( self, input_dim, output_dim):
        super( Encoder, self).__init__()

        layer_1 = nn.Sequential(
            nn.Conv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size
            )
        )
    
    def forward(self, x):
        return self.nn(x)

In [7]:
class Decoder(nn.Module):
    def __init__( self, input_dim, output_dim):
        super( Decoder, self).__init__()

        layer_2 = nn.Sequential(
            nn.ConvTranspose1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size
            )
        )
    
    def forward(self, x):
        return self.nn(x)

In [8]:
class ae_basic(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ae_basic, self).__init__()
        
        self.encoder = Encoder( input_dim, output_dim)
        self.decoder = Decoder( input_dim, output_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x 

In [25]:
class ae_tester(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ae_tester, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size
            )
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size
            )
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x

In [33]:
def train_model(model, train_set, lr, epochs):
    lr = 1e-3 if None else lr
    criterion = nn.MSELoss(size_average=False)
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(epochs):
        model.train()
        print( "something happened")

        losses = []

        for x in train_set:
            optimizer.zero_grad()

            x_prime = model(x)

            loss = criterion(x_prime, x)

            loss.backward()

            optimzer.step()

            losses.append(loss.item())


In [34]:
train_set = ecg_clean
model = ae_tester(1024, 1024)
train_model( model, train_set, None, epochs=2)

something happened


TypeError: conv1d(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [21]:
mp = model.parameters()
for i in mp:
    print("x")

In [35]:
train_set

array([[[ 0.0005712 ],
        [-0.00032728],
        [ 0.0005712 ],
        ...,
        [-0.03087534],
        [-0.02728145],
        [-0.0290784 ]],

       [[ 0.00100205],
        [ 0.00010196],
        [ 0.00100205],
        ...,
        [-0.01069912],
        [-0.00979903],
        [-0.00979903]],

       [[-0.0080301 ],
        [ 0.00454852],
        [ 0.02521339],
        ...,
        [-0.00533468],
        [-0.00443621],
        [-0.00263926]],

       ...,

       [[ 0.00203608],
        [ 0.00111272],
        [-0.00350408],
        ...,
        [-0.000734  ],
        [-0.00258072],
        [-0.00165736]],

       [[-0.00340782],
        [-0.00151746],
        [-0.00151746],
        ...,
        [-0.01096925],
        [-0.01002407],
        [-0.01380479]],

       [[-0.02242422],
        [-0.01584528],
        [-0.01396558],
        ...,
        [ 0.00671111],
        [ 0.00671111],
        [ 0.00295172]]])